In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import rasterio
from rasterio.transform import Affine

import pds4_tools

from scipy import signal
from scipy.ndimage import gaussian_filter

In [ ]:
structures = pds4_tools.read('data2015/venus_2015_north_scp.xml')
structures.info()
data = structures[0].data
dtype = data.dtype
lines = data.shape[0]
samples = data.shape[1]

In [ ]:
metadata = structures[0].meta_data                 ## currently not using metadata
label = structures.label

## upper left corner coordinates in meters
ulx = label.findall(".//cart:upperleft_corner_x")[0].text
uly = label.findall(".//cart:upperleft_corner_y")[0].text

## pixel resolution in m/pix
pix_res_x = label.findall(".//cart:pixel_resolution_x")[0].text
pix_res_y = label.findall(".//cart:pixel_resolution_y")[0].text


## projection info
map_proj_name = label.findall(".//cart:map_projection_name")[0].text
central_meridian = label.findall(".//cart:longitude_of_central_meridian")[0].text
latitude_type = label.findall(".//cart:latitude_type")[0].text
a_radius = label.findall(".//cart:a_axis_radius")[0].text.split(".")[0] + "000"       ## rdaius in xml in km; converting to m
b_radius = label.findall(".//cart:b_axis_radius")[0].text.split(".")[0] + "000"

## accessing as a dictionary - currently not used
# label_dict = label.to_dict()
# cart_dict = label_dict['Product_Observational']['Observation_Area']['Discipline_Area']['cart:Cartography']

In [ ]:
## coordinate transformation
proj4 = "+proj=sinu +lon_0=" + central_meridian + " +x_0=0 +y_0=0 +a="+ a_radius +" +b="+ b_radius + " +units=m +no_defs"
transform = Affine(float(pix_res_x), 0, float(ulx), 0, -float(pix_res_y), float(uly))

### Affine transform arguments in rasterio
If you're coming from the matrix algebra perspective, you can ignore the constants in the affine matrix and refer to the the six paramters as a, b, c, d, e, f. This is the ordering and notation used by the affine Python library.

    a = width of a pixel
    b = row rotation (typically zero)
    c = x-coordinate of the upper-left corner of the upper-left pixel
    d = column rotation (typically zero)
    e = height of a pixel (typically negative)
    f = y-coordinate of the of the upper-left corner of the upper-left pixel


In [ ]:
outfile = "data2015/north_scp_sinu.tif"
outds = rasterio.open(outfile, 'w', driver='GTiff', 
                  height = lines, 
                  width = samples, 
                  count=1, 
                  crs = proj4, 
                  dtype = dtype,
                  transform = transform,
                  compress='lzw',
                  nodata = 9999)
outds.write(data, 1)
outds.close()

In [ ]:
avgdata = np.zeros_like(data)
gaussian_filter(data, sigma=1, output=avgdata)
outfile_gau = "data2015/north_scp_sinu_sigma1.tif"
outds_gau = rasterio.open(outfile_gau, 'w', driver='GTiff', 
                  height = lines, 
                  width = samples, 
                  count=1, 
                  crs = proj4, 
                  dtype = dtype,
                  transform = transform,
                  compress='lzw',
                  nodata = 9999)
outds_gau.write(avgdata, 1)
outds_gau.close()

In [ ]:
## Reading Arecibo images without PDS4
# lines = 12000
# samples = 20000
# filename = 'data2020/venus_2020_north_inc.img'
# datatype = np.float32
# f = open(filename, 'rb')
# data_raw = np.fromfile(f, datatype)
# data = data_raw.reshape(lines, samples)